Image ECG originale → 
Crop signal → 
Grayscale → 
Otsu Threshold → 
Morph. Opening → 
Dilation → 
Erosion → 
Inversion → 
Final preprocessed image

In [5]:
#Imports & Directories
import os
import cv2
import numpy as np
from pathlib import Path

# Directories
CROPPED_DIR = Path("../Datasets/Dataset_cropped")      # Output of crop notebook
OUTPUT_DIR = Path("../Datasets/Dataset_background_black")     # Final dataset
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [6]:
def advanced_preprocessing(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None

    # 1. Bilateral filter
    smoothed = cv2.bilateralFilter(img, 9, 75, 75)

    # 2. Otsu threshold
    _, binary = cv2.threshold(smoothed, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 3. Connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
    cleaned = np.zeros_like(binary)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= 50:
            cleaned[labels == i] = 255

    # 4. Morphological closing
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    closed = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel)

    # 5. Inversion
    inverted = cv2.bitwise_not(closed)

    return inverted


In [7]:
def process_dataset():
    for class_name in os.listdir(CROPPED_DIR):
        class_path = os.path.join(CROPPED_DIR, class_name)
        if not os.path.isdir(class_path):
            continue
        
        output_class_dir = os.path.join(OUTPUT_DIR, class_name)
        os.makedirs(output_class_dir, exist_ok=True)

        print(f"Processing class: {class_name}")

        for filename in os.listdir(class_path):
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                
                img_path = os.path.join(class_path, filename)
                
                processed_img = advanced_preprocessing(img_path)

                if processed_img is not None:
                    save_path = os.path.join(output_class_dir, filename)
                    cv2.imwrite(save_path, processed_img)

    print("Preprocessing complete. Dataset created!")

In [8]:
#   Run processing
if __name__ == "__main__":
    process_dataset()


Processing class: ECG Images of Myocardial Infarction Patients
Processing class: ECG Images of Patient that have abnormal heartbeat
Processing class: ECG Images of Patient that have History of MI
Processing class: Normal Person ECG Images
Preprocessing complete. Dataset created!
